### Tesi M.N.A

#### Importing tools and functions

In [ ]:
from Tools import *

#### loading data

In [ ]:
rock_cropout, mean_elev, stazioni, unconsolidated, rocks, coastline = load_data()

#### get elevations of rock_cropouts

In [ ]:
rock_cropout['elevation'] = get_elevation(rock_cropout, mean_elev)

#### get basemap projections of all the data we want to visualize

In [ ]:
viz_cropout  = get_basemap_projection(rock_cropout.geometry)
viz_stazioni = get_basemap_projection(stazioni.geometry)
viz_rocks    = get_basemap_projection(rocks.geometry)
viz_uncons   = get_basemap_projection(unconsolidated.geometry)

#### Visualize initial data

In [ ]:
### TODO MERGE 3 GRAPH in one row

In [ ]:
m = viz_init()

m.scatter(viz_cropout.x, viz_cropout.y, c=np.log(rock_cropout.to_crs('epsg:3031')['geometry'].area), cmap=plt.cm.bwr, zorder = 3, s=0.1)
plt.colorbar(pad = 0.01 , shrink = 0.7, aspect = 50)
plt.show()

In [ ]:
m = viz_init()

m.scatter(viz_cropout.x, viz_cropout.y, c=rock_cropout['elevation'], cmap=plt.cm.bwr, zorder = 3, s=0.1)
plt.colorbar(pad = 0.01 , shrink = 0.7, aspect = 50)
m.scatter(viz_stazioni.x, viz_stazioni.y, c='black', zorder = 3, s=50)
plt.show()

In [ ]:
m = viz_init()

m.scatter(rocks_x, rocks_y, c='gray', zorder = 3, s=0.1)
m.scatter(unco_x, unco_y, c='yellow', zorder = 3, s=0.1)
plt.show()

#### Get clustering features

In [ ]:
cluster_data = scale_data(get_cluster_data(rock_cropout))

In [ ]:
def appendCluster(clusters, point):
        for i in range(num_cluster):
            if clusters.geometry[i].contains(point):
                return i
        return -1

def getElevRange():
    elevRange = []
    for i in range(num_cluster):
        temp = rock_cropout[rock_cropout.cluster == i]
        elevRange.append(np.max(temp.elev) - np.min(temp.elev))
    return pd.Series(elevRange)

num_cluster = 1

def getClusters(n):
    global num_cluster
    num_cluster = n
    kmeans = KMeans(n_clusters=num_cluster, init="k-means++",n_init=10,tol=1e-04, random_state=42)
    kmeans.fit(X)
    
    m = Basemap(projection='spstere',boundinglat=-60,lon_0=180,resolution='h') 
    
    clusterDF['label']=kmeans.labels_
    clusters = pd.DataFrame(columns=('cluster_num','geometry'))    
    for label in range(0,num_cluster):
        arr = clusterDF[clusterDF.label == label][['lon','lat']].to_numpy()
        hull = ch(arr)
        polylist = []
        for idx in hull.vertices: #Indices of points forming the vertices of the convex hull.
            polylist.append(arr[idx]) #Append this index point to list
        p = Polygon(polylist)
        clusters.at[label] = [label,p]
    clusters = gpd.GeoDataFrame(clusters, crs='EPSG:{}'.format(3031), geometry='geometry')
    clusters['centroid'] = clusters.geometry.apply(lambda x: x.centroid)
    CCxpt, CCypt = m(clusters.centroid.to_crs('epsg:4326').x, clusters.centroid.to_crs('epsg:4326').y)
    rock_cropout['cluster'] = rock_cropout.geometry.apply(lambda x: appendCluster(clusters, x.centroid))
    
    patches = []
    for poly in clusters.to_crs('epsg:4326').geometry:
        if poly.geom_type == 'Polygon':
            mpoly = shapely.ops.transform(m, poly)
            patches.append(PolygonPatch(mpoly))
        elif poly.geom_type == 'MultiPolygon':
            for subpoly in poly:
                mpoly = shapely.ops.transform(m, poly)
                patches.append(PolygonPatch(mpoly))
        else:
            print(poly, 'is neither a polygon nor a multi-polygon. Skipping it')
            
    clusters['elev'] = rock_cropout.groupby('cluster').elev.mean().reset_index().sort_values(by='cluster').iloc[1:].elev.values
    clusters['elevation_range'] = getElevRange()
    
    #clusters = clusters.to_crs('EPSG:4326')
    
    fig, ax1 = plt.subplots(figsize=(20, 20))
    m.drawcoastlines()
    m.fillcontinents(color='white',lake_color='aqua')
    m.drawmapboundary(fill_color='lightblue')
    
    # Cropout
    # clust_x, clust_y = m(clusterDF.to_crs('epsg:4326').geometry.x,clusterDF.to_crs('epsg:4326').geometry.y)
    # Dovrei poter usare le stesse proiezioni di cropout essendo gli stessi 
    # punti passati a clusterDF, provare a sostituire durante il refactoring
    
    m.scatter(clust_x, clust_y, c=kmeans.labels_, zorder = 3, s=0.1)
    p = PatchCollection(patches, cmap=matplotlib.cm.jet, match_original=True, zorder=4, alpha=0.5)
    p.set_array(clusters.elev)
    ax1.add_collection(p)
    plt.colorbar(p, pad = 0.01 , shrink = 0.85, aspect = 20)
    m.scatter(CCxpt, CCypt, marker='P', c='white', zorder = 4, s=100)
    m.scatter(sxpt, sypt, marker='^', c='black', zorder = 4, s=150)
    plt.show()
    return clusters

In [ ]:
clusters = getClusters(8)

In [ ]:
distance_from_coastline = []
for point in range(len(stazioni)):
    p1 = stazioni.iloc[point].geometry
    p2 = coastline.iloc[np.argmin([stazioni.iloc[point].geometry.distance(line) for line in coastline.geometry])].geometry

    ax = coastline.plot()
    ax.scatter(p1.x, p1.y, c='black', zorder=3)
    ax.scatter(p2.centroid.x, p2.centroid.y, c='red', zorder=3)
    distance = p1.distance(p2) / 1000
    distance_from_coastline.append(distance)
    print(distance)
    plt.show()

In [ ]:
stazioni['distance_from_coastline(km)'] = np.around(distance_from_coastline,2)

In [ ]:
def getArea(poly):
    geom = poly
    geom_area = ops.transform(
        partial(
            pyproj.transform,
            pyproj.Proj(init='EPSG:4326'),
            pyproj.Proj(
                proj='aea',
                lat_1=geom.bounds[1],
                lat_2=geom.bounds[3]
            )
        ),
        geom)
    return round(geom_area.area / 1e6)

In [ ]:
clusters['area_km2'] = clusters.to_crs('epsg:4326').geometry.apply(lambda x: getArea(x))

In [ ]:
fig = px.scatter(clusters, x="cluster_num", y="elev", size='area_km2', color='elevation_range',width=1000, height=500)
fig.show()

In [ ]:
%%time
def get_total_cropout_area_under_radius(point, target, radius):
    arr = [point.distance(geom) / 1000 for geom in target]
    idx = [idx for idx in range(len(arr)) if arr[idx] < radius]
    return np.around(target.iloc[idx].geometry.area.values.sum()/1e6,2)

stazioni['nearest_cluster']             = stazioni.geometry.apply(lambda x: clusters.iloc[np.argmin([x.distance(geom.centroid) / 1000 for geom in clusters.geometry])].cluster_num )
stazioni['distance_to_nearest_cluster'] = stazioni.geometry.apply(lambda x: np.around(x.distance(clusters.iloc[np.argmin([x.distance(geom.centroid) / 1000 for geom in clusters.geometry])].centroid) / 1000,2 ))
stazioni['nearest_cropout(km)']         = stazioni.geometry.apply(lambda x: np.around(np.min([x.distance(geom) / 1000 for geom in rock_cropout.geometry]), 2) )
stazioni['cropout_area_within_500km']   = stazioni.geometry.apply(lambda x: get_total_cropout_area_under_radius(x, rock_cropout.geometry, 500))
stazioni['cropout_area_within_1000km']  = stazioni.geometry.apply(lambda x: get_total_cropout_area_under_radius(x, rock_cropout.geometry, 1000))

In [ ]:
fig = px.scatter(stazioni, x="distance_from_coastline(km)", y="distance_to_nearest_cluster", size='cropout_area_within_1000km', color='dust_conc_(ppb)',width=1000, height=500)
fig.show()

In [ ]:
fig = px.scatter(stazioni, x="dust_conc_(ppb)", y="cropout_area_within_1000km", size='distance_from_coastline(km)', color='altitude',width=1000, height=500)
fig.show()